In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [3]:
d_fp_arr = np.genfromtxt('direct_fp_arr', delimiter=',')
in_fp_arr = np.genfromtxt('indirect_fp_arr', delimiter=',')
m_fp_arr = np.genfromtxt('metallic_fp_arr', delimiter=',')

In [4]:
d_fp_arr.shape, in_fp_arr.shape, m_fp_arr.shape

((48, 300), (12720, 300), (312, 300))

In [5]:
d_fp_arr[0, :]

array([1.32151946e+00, 5.21004583e-01, 5.17490732e-01, 4.84209395e-01,
       3.01039397e-01, 1.47388537e-01, 1.45111664e-01, 1.17087873e-01,
       1.13405364e-01, 9.86005220e-02, 8.73598024e-02, 7.98760930e-02,
       7.64476850e-02, 7.60371812e-02, 6.18558480e-02, 5.61245247e-02,
       3.97216136e-02, 3.74927633e-02, 2.58488294e-02, 2.56525486e-02,
       2.08963734e-02, 1.95846546e-02, 1.93028413e-02, 1.27966116e-03,
       1.18343539e-03, 1.18280578e-03, 6.39563867e-05, 6.20779403e-05,
       6.01975569e-05, 5.90765884e-05, 5.32111639e-05, 5.15338281e-05,
       2.75438377e-05, 2.50999073e-05, 1.09704445e-05, 1.09631648e-05,
       3.85474003e-06, 3.82914628e-06, 1.88284877e-06, 7.90168512e-07,
       7.26134191e-07, 2.78813977e-07, 2.35246273e-07, 2.26400960e-07,
       2.26089642e-07, 1.98102526e-09, 1.97964514e-09, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [6]:
d_fp_list = d_fp_arr.ravel().tolist()
in_fp_list = in_fp_arr.ravel().tolist()
m_fp_list = m_fp_arr.ravel().tolist()


In [7]:
len(d_fp_list)

14400

In [8]:
d_fp_flat = np.array(d_fp_list).reshape((int(len(d_fp_arr)/12)), 3600)
in_fp_flat = np.array(in_fp_list).reshape((int(len(in_fp_arr)/12)), 3600)
m_fp_flat = np.array(m_fp_list).reshape((int(len(m_fp_arr)/12)), 3600)

In [9]:
FP_ARR = np.vstack((d_fp_flat, in_fp_flat, m_fp_flat))

In [10]:
d_fp_flat.shape, in_fp_flat.shape, m_fp_flat.shape, FP_ARR.shape

((4, 3600), (1060, 3600), (26, 3600), (1090, 3600))

In [11]:
Y_ARR = [0]*4 + [1]*1060 + [2]*26

In [12]:
len(Y_ARR)

1090

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    FP_ARR, Y_ARR, test_size = 0.2, random_state = 42 )
x_min, x_max = FP_ARR[:, 0].min() - 0.5, FP_ARR[:, 0].max() + 0.5
y_min, y_max = FP_ARR[:, 1].min() - 0.5, FP_ARR[:, 1].max() + 0.5

In [14]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

for name, clf in zip(names, classifiers):
    # ds_cnt = 0 
    # ax = plt.subplot(1, len(classifiers) + 1, 1)
    clf = make_pipeline(StandardScaler(), clf)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(name, score)
    '''
    DecisionBoundaryDisplay.from_estimator(
        clf, FP_ARR, cmap=plt.cm.RdBu, alpha=0.8, ax=ax, eps=0.5
    )
    # Plot the training points
    ax.scatter(
        X_train[:, 0], X_train[:, 1], c=y_train, cmap=ListedColormap(["#FF0000", "#0000FF"]), edgecolors="k"
    )
    # Plot the testing points
    ax.scatter(
        X_test[:, 0],
        X_test[:, 1],
        c=y_test,
        cmap=ListedColormap(["#FF0000", "#0000FF"]),
        edgecolors="k",
        alpha=0.6,
    )
    
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())
    if ds_cnt == 0:
        ax.set_title(name)
    ax.text(
        x_max - 0.3,
        y_min + 0.3,
        ("%.2f" % score).lstrip("0"),
        size=15,
        horizontalalignment="right",
    )
    '''

Nearest Neighbors 0.981651376146789
Linear SVM 0.981651376146789
RBF SVM 0.981651376146789
Gaussian Process 0.981651376146789
Decision Tree 0.981651376146789
Random Forest 0.981651376146789
Neural Net 0.981651376146789
AdaBoost 0.981651376146789
Naive Bayes 0.009174311926605505


/Users/shuosmacmini/miniforge3/envs/jupyter/lib/python3.9/site-packages/sklearn/naive_bayes.py:515: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/Users/shuosmacmini/miniforge3/envs/jupyter/lib/python3.9/site-packages/sklearn/naive_bayes.py:516: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/Users/shuosmacmini/miniforge3/envs/jupyter/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QDA 0.009174311926605505


/Users/shuosmacmini/miniforge3/envs/jupyter/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:951: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Users/shuosmacmini/miniforge3/envs/jupyter/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:951: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/Users/shuosmacmini/miniforge3/envs/jupyter/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:954: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
